# Скачивание файлов близкородственной бактерии

In [1]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "export PATH=\${PATH}:/root/edirect" >> ${HOME}/.bashrc

or manually edit the PATH variable assignment in your .bashrc file.

Would you like to do that automatically now? [y/N]
y
OK, done.

To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



In [2]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

# Установка програм 

In [3]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 4.4 MB/s 


In [4]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [95.2 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [983 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports In

In [5]:
!apt-get install ncbi-blast+

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  ncbi-data
The following NEW packages will be installed:
  ncbi-blast+ ncbi-data
0 upgraded, 2 newly installed, 0 to remove and 38 not upgraded.
Need to get 13.1 MB of archives.
After this operation, 66.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 ncbi-data all 6.1.20170106-2 [3,645 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 ncbi-blast+ amd64 2.6.0-1 [9,446 kB]
Fetched 13.1 MB in 3s (5,118 kB/s)
Selecting previously unselected package ncbi-data.
(Reading database ... 123941 files and directories currently installed.)
Preparing to unpack .../ncbi-data_6.1.20170106-2_all.deb ...
Unpacking ncbi-data (6.1.20170106-2) ...
Selectin

In [6]:
!blastn -help

USAGE
  blastn [-h] [-help] [-import_search_strategy filename]
    [-export_search_strategy filename] [-task task_name] [-db database_name]
    [-dbsize num_letters] [-gilist filename] [-seqidlist filename]
    [-negative_gilist filename] [-entrez_query entrez_query]
    [-db_soft_mask filtering_algorithm] [-db_hard_mask filtering_algorithm]
    [-subject subject_input_file] [-subject_loc range] [-query input_file]
    [-out output_file] [-evalue evalue] [-word_size int_value]
    [-gapopen open_penalty] [-gapextend extend_penalty]
    [-perc_identity float_value] [-qcov_hsp_perc float_value]
    [-max_hsps int_value] [-xdrop_ungap float_value] [-xdrop_gap float_value]
    [-xdrop_gap_final float_value] [-searchsp int_value]
    [-sum_stats bool_value] [-penalty penalty] [-reward reward] [-no_greedy]
    [-min_raw_gapped_score int_value] [-template_type type]
    [-template_length int_value] [-dust DUST_options]
    [-filtering_db filtering_database]
    [-window_masker_taxid window_ma

Прочитать подробнее можно тут https://www.ncbi.nlm.nih.gov/books/NBK279690/ 

# Предсказание рибосомальной рнк

In [7]:
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
import re
import pandas as pd

In [8]:
#Находим координаты начала, конца, стренд рРНК у близкородственной бактерии
coords = []
for feature in SeqIO.read("T_oleivorans_MIL_1.gbk", "genbank").features:
  if feature.type == 'rRNA':
    coords.append([int(feature.location.start), int(feature.location.end), feature.strand])

In [9]:
#Находим последовательности рРНК, которые находятся по полученным координатам
sequences = SeqIO.read("T_oleivorans_MIL_1.gbk", "genbank").seq
sequencesRNA = []
for coord in coords:
  if coord[2] == 1:
    sequencesRNA.append([coord[0],coord[1],str(sequences[coord[0]:coord[1]]),'plus'])
  else:
    sequencesRNA.append([coord[0],coord[1],str(sequences[coord[0]:coord[1]]),'minus'])

In [10]:
#Записываем полученные последовательности в fasta файл
with open('rRNA.fasta','w') as file:
  for sequence in sequencesRNA:
    file.write(f'>rRNA_{sequence[0]}...{sequence[1]}\n')
    file.write(sequence[2] + '\n')

In [11]:
#Скачиваем файл скаффолдов
!wget https://raw.githubusercontent.com/miamib34ch/hse22_hw2/main/data/scaffolds.fa

--2022-10-19 20:17:21--  https://raw.githubusercontent.com/miamib34ch/hse22_hw2/main/data/scaffolds.fa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3965905 (3.8M) [text/plain]
Saving to: ‘scaffolds.fa’

scaffolds.fa        100%[===================>]   3.78M  --.-KB/s    in 0.09s   

2022-10-19 20:17:23 (43.8 MB/s) - ‘scaffolds.fa’ saved [3965905/3965905]



In [13]:
#Делаем выравнивание 
!blastn -query rRNA.fasta -subject scaffolds.fa -outfmt '10 qseqid sseqid pident sstrand evalue' > allignment_stats.txt
!blastn -query rRNA.fasta -subject scaffolds.fa -outfmt 7

# BLASTN 2.6.0+
# Query: rRNA_341494...343033
# Database: User specified sequence set (Input: scaffolds.fa)
# Fields: query acc.ver, subject acc.ver, % identity, alignment length, mismatches, gap opens, q. start, q. end, s. start, s. end, evalue, bit score
# 4 hits found
rRNA_341494...343033	scaffold1_cov231	100.000	1539	0	0	1	1539	2527358	2528896	0.0	2843
rRNA_341494...343033	scaffold1_cov231	99.935	1539	1	0	1	1539	846845	845307	0.0	2837
rRNA_341494...343033	scaffold1_cov231	100.000	64	0	0	1	64	1673342	1673405	7.22e-27	119
rRNA_341494...343033	scaffold32_cov590	100.000	1399	0	0	134	1532	1399	1	0.0	2584
# BLASTN 2.6.0+
# Query: rRNA_343487...346374
# Database: User specified sequence set (Input: scaffolds.fa)
# Fields: query acc.ver, subject acc.ver, % identity, alignment length, mismatches, gap opens, q. start, q. end, s. start, s. end, evalue, bit score
# 5 hits found
rRNA_343487...346374	scaffold1_cov231	100.000	2887	0	0	1	2887	2529147	2532033	0.0	5332
rRNA_343487...346374	scaffold1

In [14]:
#Возьмём с самым низким evalue из каждого промежутка координат
names = ['qseqid', 'sseqid', '% identity', 'sstrand', 'evalue']
stats = pd.read_csv('allignment_stats.txt', sep=',', header=None, names=names)
stats = stats.sort_values('evalue', ascending=True).drop_duplicates('qseqid')
stats

,qseqid,sseqid,% identity,sstrand,evalue
0,rRNA_341494...343033,scaffold1_cov231,100.000,plus,0.000000e+00
22,rRNA_2583918...2585457,scaffold1_cov231,99.935,plus,0.000000e+00
30,rRNA_3418942...3421829,scaffold1_cov231,100.000,minus,0.000000e+00
35,rRNA_3422079...3423618,scaffold1_cov231,99.935,plus,0.000000e+00
43,rRNA_3424290...3427177,scaffold1_cov231,100.000,minus,0.000000e+00
48,rRNA_3427427...3428966,scaffold1_cov231,99.935,plus,0.000000e+00
20,rRNA_2580781...2583668,scaffold41_cov598,100.000,plus,0.000000e+00
4,rRNA_343487...346374,scaffold1_cov231,100.000,plus,0.000000e+00
39,rRNA_3423993...3424109,scaffold1_cov231,100.000,plus,5.670000e-57
27,rRNA_3418645...3418761,scaffold1_cov231,100.000,minus,5.670000e-57


In [15]:
#Оставим только строки, со стрендами, которые совпадают со стрендами близкородственной бактерии
for sequence in sequencesRNA:
  for line in stats.iterrows():
    if line[1][0] == f'rRNA_{sequence[0]}...{sequence[1]}' and sequence[3] != line[1][3]:
      stats = stats.drop(line[0])
stats

,qseqid,sseqid,% identity,sstrand,evalue
0,rRNA_341494...343033,scaffold1_cov231,100.000,plus,0.000000e+00
30,rRNA_3418942...3421829,scaffold1_cov231,100.000,minus,0.000000e+00
43,rRNA_3424290...3427177,scaffold1_cov231,100.000,minus,0.000000e+00
4,rRNA_343487...346374,scaffold1_cov231,100.000,plus,0.000000e+00
27,rRNA_3418645...3418761,scaffold1_cov231,100.000,minus,5.670000e-57
14,rRNA_2580484...2580600,scaffold1_cov231,98.276,minus,1.230000e-53


In [16]:
#Скачиваем файл полученный в основной части, чтобы дополнить аннотацию
!wget https://raw.githubusercontent.com/miamib34ch/hse22_hw2/main/data/GENOME.gbk
genome = SeqIO.to_dict(SeqIO.parse("/content/GENOME.gbk", "genbank"))

--2022-10-19 20:20:08--  https://raw.githubusercontent.com/miamib34ch/hse22_hw2/main/data/GENOME.gbk
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7346795 (7.0M) [text/plain]
Saving to: ‘GENOME.gbk’

GENOME.gbk          100%[===================>]   7.01M  --.-KB/s    in 0.1s    

2022-10-19 20:20:10 (68.0 MB/s) - ‘GENOME.gbk’ saved [7346795/7346795]



In [17]:
#Разбиваем полученные данные для более удобной работы с ними
stats['Start'] = 0
stats['End'] = 0

for line in stats.iterrows():
  cords = stats.at[line[0],"qseqid"].split('...')
  stats.at[line[0],"Start"] = cords[0].split('_')[1]
  stats.at[line[0],"End"] = cords[1]
  if stats.at[line[0],"sstrand"] == 'minus':
    stats.at[line[0],"sstrand"] = '-'
  else: 
    stats.at[line[0],"sstrand"] = '+'

stats

,qseqid,sseqid,% identity,sstrand,evalue,Start,End
0,rRNA_341494...343033,scaffold1_cov231,100.000,+,0.000000e+00,341494,343033
30,rRNA_3418942...3421829,scaffold1_cov231,100.000,-,0.000000e+00,3418942,3421829
43,rRNA_3424290...3427177,scaffold1_cov231,100.000,-,0.000000e+00,3424290,3427177
4,rRNA_343487...346374,scaffold1_cov231,100.000,+,0.000000e+00,343487,346374
27,rRNA_3418645...3418761,scaffold1_cov231,100.000,-,5.670000e-57,3418645,3418761
14,rRNA_2580484...2580600,scaffold1_cov231,98.276,-,1.230000e-53,2580484,2580600


In [18]:
#переделываем аннотацию
for id in genome:
  for feature in genome[id].features:
    for line in stats.iterrows():
      if (str(feature.location) == f"[{line[1][5]}:{line[1][6]}]({line[1][3]})"):
        genome[id].annotations['molecule_type'] = 'rRna'

In [19]:
#записываем результаты в генбанковский файл
SeqIO.write(genome.values(), "GENOME.gbk", "genbank")

69